# Generic Programming

## Example: Vandermonde matrix
\begin{align}V=\begin{bmatrix}1&\alpha _{1}&\alpha _{1}^{2}&\dots &\alpha _{1}^{n-1}\\1&\alpha _{2}&\alpha _{2}^{2}&\dots &\alpha _{2}^{n-1}\\1&\alpha _{3}&\alpha _{3}^{2}&\dots &\alpha _{3}^{n-1}\\\vdots &\vdots &\vdots &\ddots &\vdots \\1&\alpha _{m}&\alpha _{m}^{2}&\dots &\alpha _{m}^{n-1}\end{bmatrix}\end{align}

Here is a straightforward Julia implementation

In [ ]:
function vander_naive(x)
    m = length(x)
    V = zeros(m,m)

    # first column
    for j in 1:m
        V[j,1] = 1.0
    end

    # all other columns
    for i in 2:m
        for j in 1:m
            V[j,i] = x[j] * V[j,i-1]
        end
    end
    return V
end

In [ ]:
vander_naive(1:3)

In [ ]:
function vander(x::AbstractVector{T}) where T
    m = length(x)
    V = Matrix{T}(undef, m, m) # not zeros(m,m)

    # first column
    for j in 1:m
        V[j,1] = one(T) # not 1.0
    end

    # all other columns
    for i in 2:m
        for j in 1:m
            V[j,i] = x[j] * V[j,i-1]
        end
    end
    return V
end

In [ ]:
vander(1:3)

In [ ]:
vander([1.0, 2.0, 3.0])

### Python / numpy

<p><img src="./imgs/numpy_vander.png" alt="drawing" width="1500"/></p>

(The source code for this function is [here](https://github.com/numpy/numpy/blob/v1.16.1/numpy/lib/twodim_base.py#L475-L563). It calls `np.multiply.accumulate` which is implemented in C [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/ufunc_object.c#L3678). However, this code doesn't actually perform the computation, it basically only checks types and stuff. The actual kernel that gets called is [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/loops.c.src#L1742). This isn't even C code but a template for C code which is used to generate type specific kernels.)

### Quick speed comparison

<img src="./imgs/vandermonde.svg" alt="drawing" width="600"/>

Note that the clean and concise Julia implementation is **beating numpy's C implementation for small matrices** and is **on-par for large matrix sizes**.

### Genericity: Beyond numbers

Our Julia code is ***generic*** and works for ***any element type `T`*** that implements

* `*` (multiplication)
* `one(T)` (identity under multiplication)

The type doesn't even have to be numeric!

In [ ]:
vander(["A", "B", "C", "D"])

Let's do something more useful:

In [ ]:
using Symbolics

In [ ]:
@variables a b c d e

In [ ]:
typeof(a)

In [ ]:
one(a)

In [ ]:
a * a

In [ ]:
v = vander([a,b,c,d,e])

In [ ]:
substitute(v, Dict(b => 2, d => 4))

## "Emergent" Features: Differential equation solving with uncertainty

In [ ]:
using OrdinaryDiffEqTsit5, Measurements, Plots

#Half-life of Carbon-14 is 5730 years.
c = 5.730 ± 2

#Setup
u0 = 1.0 ± 0.1
tspan = (0.0, 1.0)

#Define the problem
radioactivedecay(u,p,t) = -c*u

#Pass to solver
prob = ODEProblem(radioactivedecay,u0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

plot(sol.t, sol.u, ylabel="u(t)", xlabel="t", lw=2, legend=false, frame=:box)

**Historical note**: In some sense, **Julia implemented that feature by itself**. The authors of Measurements.jl and DifferentialEquations.jl [never had any collabration on this](https://discourse.julialang.org/t/differentialequations-jl-and-measurements-jl/6350).